# SCD Type 2

SCD Type 2 is a technique used to preserve full history of changes to dimension attributes. Instead of overwriting existing records, a new row is inserted whenever a change occurs, and the previous row is marked as inactive using fields like end_date, is_current, or a version number. This allows accurate historical reporting and time-based analysis.

In [0]:
%sql
create database scd;

We will have similar tables as in SCD type1, but the dim table will have start date , end date and a flag as below.

In [0]:
%sql
create table employee_stg (employee_id int primary key, name string, salary int, dept string);
create table employee (emp_surrogate_key bigint GENERATED ALWAYS AS identity ,employee_id int , name string, salary int, dept string, start_date date, end_date date, is_active string);

we will maintain intial data as below in dim table

In [0]:
%sql
insert into employee (employee_id, name, salary, dept, start_date, end_date, is_active) values
  (1,'Sanjay','30000','Sales','2000-10-10','9999-12-31','Y'),
  (2,'Vijay','45000','Sales','2019-04-03','9999-12-31','Y'),
  (3,'Akash','43000','IT','2017-09-12','9999-12-31','Y'),
  (4,'nakul','60000','Sales','2019-12-12','9999-12-31','Y'),
  (5,'preethi','55000','OP','1996-01-09','9999-12-31','Y'),
  (6,'hari','33000','OP','2003-11-04','9999-12-31','Y')

In [0]:
%sql
select * from employee

![image_1770033157239.png](./image/image_1770033157239.png "image_1770033157239.png")

Now lets populate the stg table with one new record and 2 updated records.

In [0]:
%sql
insert into employee_stg values
(1,'Sanjay','60000','IT'),
(2,'Vijay','55000','Admin'),
(7,'naveen','93000','Dev')

In [0]:
%sql
select * from employee_stg

![image_1770035711207.png](./image/image_1770035711207.png "image_1770035711207.png")

Here we will mark the existing records in Dim table as inactive whose ID is matching with Staging table records.

In [0]:
%sql
merge into employee as dim
using employee_stg as stg
ON dim.employee_id = stg.employee_id and dim.is_active='Y'
when matched then
  update set dim.end_date = current_date, dim.is_active = 'N'

Below it can be seen that employee with id 1,2 are made inactive

In [0]:
%sql
select * from employee

![image_1770036970325.png](./image/image_1770036970325.png "image_1770036970325.png")

Now lets insert the records from stg to dim table as active records.

In [0]:
%sql
insert into employee (employee_id, name, salary, dept, start_date, end_date, is_active)
Select employee_id, name, salary, dept, current_date as start_date, '9999-12-31' as end_date,'Y' as is_active 
from employee_stg

In [0]:
%sql
select * from employee

Now in the below , we cam see that the new entry has been added to employee dimension also the existing employee are inseert back with new surrogate_key, one more things to keep in mind that, in dim employee table we cannot use employee id as unique key, surrogate_key will be unique. if its necessary we can use employee id and start date as composite key

![image_1770036020353.png](./image/image_1770036020353.png "image_1770036020353.png")